In [83]:
import pandas as pd
import numpy as np
import math
from pathlib import Path

In [84]:
BASE = Path("Datasets/mockup_ver2/")

products   = pd.read_csv(BASE/"products.csv")
promotions = pd.read_csv(BASE/"promotions.csv", parse_dates=["start_date","end_date"])
promo_tx   = pd.read_csv(BASE/"promotion_transactions.csv")
stores     = pd.read_csv(BASE/"stores_with_patterns.csv")
tx         = pd.read_csv(BASE/"transactions.csv", parse_dates=["timestamp"])
users      = pd.read_csv(BASE/"users.csv")  # เผื่อไว้ แม้จะมีแค่ user_id
festivals  = pd.read_csv(BASE/"festivals_2025.csv")

tx_merge = pd.read_csv(BASE/"tx_merge1.csv")  # from previous notebook

C:\Users\User\AppData\Local\Temp\ipykernel_28604\2980159663.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  promotions = pd.read_csv(BASE/"promotions.csv", parse_dates=["start_date","end_date"])
C:\Users\User\AppData\Local\Temp\ipykernel_28604\2980159663.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  promotions = pd.read_csv(BASE/"promotions.csv", parse_dates=["start_date","end_date"])
C:\Users\User\AppData\Local\Temp\ipykernel_28604\2980159663.py:7: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  tx         = pd.read_csv(BASE/"transactions.csv", parse_dates=["timestamp"])


In [85]:
tx_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20076 entries, 0 to 20075
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   transaction_id       19178 non-null  object 
 1   user_id              19178 non-null  object 
 2   product_id           19178 non-null  object 
 3   qty                  19178 non-null  float64
 4   price                19178 non-null  float64
 5   timestamp            19178 non-null  object 
 6   store_id             19178 non-null  object 
 7   is_online            19178 non-null  object 
 8   products.category    19178 non-null  object 
 9   products.brand       19178 non-null  object 
 10  products.base_price  19178 non-null  float64
 11  stores.zone          19178 non-null  float64
 12  stores.province      19178 non-null  object 
 13  stores.profile       19178 non-null  object 
 14  promo_id             5076 non-null   object 
dtypes: float64(4), object(11)
memory usa

## Time Feature

In [ ]:
tx_merge["timestamp"] = pd.to_datetime(
    tx_merge["timestamp"], 
    format="%d/%m/%Y %H:%M"   # รูปแบบวันที่-เวลาในไฟล์
)

tx_merge["order_date"] = tx_merge["timestamp"].dt.date
tx_merge["order_hour"] = tx_merge["timestamp"].dt.hour
tx_merge["dayofweek"] = tx_merge["timestamp"].dt.dayofweek
tx_merge["month"] = tx_merge["timestamp"].dt.month
tx_merge["day"] = tx_merge["timestamp"].dt.day
tx_merge["weekofyear"] = tx_merge["timestamp"].dt.isocalendar().week
tx_merge["quarter"] = tx_merge["timestamp"].dt.quarter
tx_merge["is_weekend"] = tx_merge["dayofweek"].apply(lambda x: 1 if x >= 5 else 0)

tx_merge["store_id"] = tx_merge["store_id"].astype(str).str.strip()

def month_to_th_season(m):
    if pd.isna(m):
        return np.nan
    m = int(m)
    if m in (3,4,5):
        return "Summer"
    elif m in (6,7,8,9,10):
        return "Rainy"
    else:  # 11,12,1,2
        return "Winter"

tx_merge["thai_season"] = tx_merge["month"].apply(month_to_th_season)

stores["store_id"] = stores["store_id"].astype(str).str.strip()


In [87]:
festivals = festivals.rename(columns={"category":"festival_category"})

# fix NaT
for c in ["start_date","end_date"]:
    festivals[c] = (festivals[c].astype(str)
                   .str.replace("\u200b","", regex=False)  # zero-width space
                   .str.strip()
                   .replace({"": None, "NULL": None, "NaN": None}))
    
    
festivals["start_date"] = pd.to_datetime(festivals["start_date"], errors="coerce")
festivals["end_date"]   = pd.to_datetime(festivals["end_date"], errors="coerce")
festivals["date"] = [pd.date_range(s.normalize(), e.normalize(), freq="D") for s, e in zip(festivals["start_date"], festivals["end_date"])]
fest_daily = festivals.explode("date", ignore_index=True)
fest_daily["date"] = fest_daily["date"].dt.date
festivals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   festival_name      29 non-null     object        
 1   start_date         29 non-null     datetime64[ns]
 2   end_date           29 non-null     datetime64[ns]
 3   festival_category  29 non-null     object        
 4   date               29 non-null     object        
dtypes: datetime64[ns](2), object(3)
memory usage: 1.3+ KB


In [88]:
tx_merge = tx_merge.merge(
    fest_daily[["date","festival_name","festival_category"]],
    left_on="order_date", right_on="date", how="left"
)
tx_merge["festival_name"] = tx_merge["festival_name"].fillna("").astype(str).str.strip()
tx_merge["festival_category"] = tx_merge["festival_category"].fillna("").astype(str).str.strip()
tx_merge["InFestival"] = (tx_merge["festival_name"] != "").astype(int)

In [89]:
tx_merge.head()

,transaction_id,user_id,product_id,qty,price,timestamp,store_id,is_online,products.category,products.brand,...,dayofweek,month,day,weekofyear,quarter,is_weekend,date,festival_name,festival_category,InFestival
0,TX0000510,U0038,P0442,4.0,694.0,2025-09-22 01:44:00,S042,False,Others,Brand_001,...,0.0,9.0,22.0,39,3.0,0,2025-09-22,เทศกาลกินเจ (Vegetarian Festival),เทศกาลไทย-จีน,1
1,TX0000764,U0058,P0689,4.0,514.0,2025-09-21 20:21:00,S042,False,HealthBeauty,Brand_032,...,6.0,9.0,21.0,38,3.0,1,NaN,,,0
2,TX0001136,U0086,P0078,4.0,460.0,2025-06-29 04:36:00,S042,False,DairyBakery,Brand_002,...,6.0,6.0,29.0,26,2.0,1,NaN,,,0
3,TX0001608,U0117,P0653,4.0,390.0,2025-06-25 23:37:00,S042,False,Household,Brand_012,...,2.0,6.0,25.0,26,2.0,0,NaN,,,0
4,TX0001699,U0125,P0843,4.0,462.0,2025-08-04 01:28:00,S042,False,HealthBeauty,Brand_006,...,0.0,8.0,4.0,32,3.0,0,NaN,,,0


Store Time Features

In [ ]:
hour_cols = [c for c in stores.columns if c.startswith("hour_w_")]

st_hour_long = stores.melt(
    id_vars=["store_id", "profile", "weekday_boost", "weekend_boost", "festival_boost", "peaks_encoded"],
    value_vars=hour_cols, var_name="hour_col", value_name="hour_weight"
)

st_hour_long["order_hour"] = st_hour_long["hour_col"].str.replace("hour_w_", "", regex=False).astype(int)
st_hour_long = st_hour_long.drop(columns=["hour_col"])

tx_merge = tx_merge.merge(
    st_hour_long,
    on=["store_id", "order_hour"],
    how="left",
    validate="m:1"
)

In [92]:
tx_merge.columns

Index(['transaction_id', 'user_id', 'product_id', 'qty', 'price', 'timestamp',
       'store_id', 'is_online', 'products.category', 'products.brand',
       'products.base_price', 'stores.zone', 'stores.province',
       'stores.profile', 'promo_id', 'order_date', 'order_hour', 'dayofweek',
       'month', 'day', 'weekofyear', 'quarter', 'is_weekend', 'date',
       'festival_name', 'festival_category', 'InFestival', 'profile',
       'weekday_boost', 'weekend_boost', 'festival_boost', 'peaks_encoded',
       'hour_weight'],
      dtype='object')